In [ ]:
import os
#set paths for jwst pipeline
os.environ["CRDS_PATH"] = "/home/dori/Documents/misc/JWST_STAGE2/crds_cache"
os.environ["CRDS_SERVER_URL"] = "https://jwst-crds.stsci.edu"
from astropy.io import fits
from tqdm.auto import tqdm
from jwst.pipeline import Image2Pipeline
import pipeline_1

In [ ]:
#file1 = "uncal_abdor/jw01093012001_03102_00001_nis_uncal.fits"
file2 = "uncal_abdor/jw01093015001_03102_00001_nis_uncal.fits"
#file3 = "uncal_abdor/jw01093018001_03102_00001_nis_uncal.fits"

files = [file1,file2,file3]
output_dir = "pipeline1_outputs"

for file in files:
    pl1 = pipeline_1.Detector1Pipeline()
    pl1.save_results = True
    pl1.output_dir = str(output_dir)
    pl1.ipc.skip = True  
    pl1.save_calibrated_ramp = True #save linearized ramp
    #only run pipeline up to saving linearized ramps
    pl1.lin_ramp_only = True
    pl1.from_lin_ramp = False
    pl1.run(str(file))

In [ ]:
#repeat for the other two observations
output_dir= "./pipeline1_outputs"

file_t = os.path.join(
    output_dir,
    "jw01093015001_03102_00001_nis_ramp.fits")

hdu = fits.open(file_t)
cube_t = hdu[1].data
cube_3t = hdu[3].data
cube_4t = hdu[4].data

for i in tqdm(range(1,cube_t.shape[1]+1)):

    hdu1 = fits.open(file_t)
    #insert data frames and metadata
    hdu1[1].data = cube_t[:,:i,:,:]
    hdu1[3].data = cube_3t[:,:i,:,:]
    hdu1[4].data = cube_4t[:,:i,:,:]
    
    output_dir2 = "./pipeline1_outputs_groups"

    file_t2 = os.path.join(
        output_dir2,
        "jw01093015001_03102_0000"+ str(i) +"_nis_uncal.fits")

    hdu1.writeto(file_t2,overwrite=False)
    hdu1.close()
    
    pl1 = pipeline_1.Detector1Pipeline()
    pl1.save_results = True
    pl1.output_dir = str(output_dir2)
    pl1.ipc.skip = True  
    pl1.save_calibrated_ramp = False 
    #run pipeline from linearized ramp step
    pl1.lin_ramp_only = False
    pl1.from_lin_ramp = True
    pl1.run(str(file_t2))
    
    pl2 = Image2Pipeline()
    pl2.save_results = True
    pl2.output_dir = output_dir2
    pl2.photom.skip = True
    pl2.resample.skip = True 
    fitsfile = file_t2.replace('_uncal.fits', '_rateints.fits')
    pl2.run(fitsfile)
    
    
    #actively delete files we don't need - probably a way to do this in the pipeline but oh well
    os.remove(file_t2) 
    file_t4 = os.path.join(
        output_dir2 ,
        "jw01093015001_03102_0000"+ str(i) +"_nis_rate.fits")
    os.remove(file_t4) 
    file_t5 = os.path.join(
        output_dir2 ,
        "jw01093015001_03102_0000"+ str(i) +"_nis_rateints.fits")
    os.remove(file_t5) 